In [2]:
import numpy as np
import matplotlib.pyplot as plt
import utilis as u
import importlib
import model as m 
import mlp as ml

Using TensorFlow backend.


# load data

In [3]:
filename = '/data_batch_1'
X_train, y_train,Y_train = u.load_data(filename, reshape=False, clipping=True)
meanX = np.mean(X_train,axis=1)
stdX = np.std(X_train,axis=1)
X_train = (X_train-meanX.reshape((len(meanX),1)))/stdX.reshape((len(stdX),1))

filename = '/data_batch_2'
X_val, y_val,Y_val = u.load_data(filename, reshape=False, clipping=True)
X_val = (X_val-meanX.reshape((len(meanX),1)))/stdX.reshape((len(stdX),1))

filename = '/test_batch'
X_test, y_test,Y_test = u.load_data(filename, reshape=False, clipping=True)
X_test = (X_test-meanX.reshape((len(meanX),1)))/stdX.reshape((len(stdX),1))

In [4]:
data = {'X_train':X_train, 'Y_train':Y_train, 'y_train':y_train,'X_val':X_val, 'Y_val':Y_val, 'y_val':y_val}

# test gradients

In [19]:
importlib.reload(m)
mlp = m.MLP(dimensions=[100,50,10])
P = mlp.forward(X_train[:100,:1])
mlp.compute_gradients(X_train[:100,:1], Y_train[:,:1],P)
mlp.compareGradients(X_train[:100,:1],Y_train[:,:1])

([2.175440781541359e-09, 1.7737316176163514e-10],
 [7.733062496273626e-11, 5.734093724796586e-11],
 [6.1146156729771686e-12, 4.653651386118951e-12],
 [5.2695548532349926e-12, 9.858529964601458e-12])

# train Model 1

In [27]:
importlib.reload(m)
mlp = m.MLP(lambda_=0.01)
GD_params = {"n_batch":100, "eta_min":1e-5, 'eta_max':1e-1, 'ns':500, 'n_cycles':1, 'freq':10}
mlp.cyclicLearning(data, GD_params, 'cyclic_learning_ex1', True, True)

  0%|          | 0/10 [00:00<?, ?it/s]

KeyboardInterrupt: 

# train Model 2 

In [12]:
importlib.reload(m)
mlp = m.MLP(lambda_=0.01)
GD_params = {"n_batch":100, "eta_min":1e-5, 'eta_max':1e-1, 'ns':800, 'n_cycles':3, 'freq':10}
mlp.cyclicLearning(data, GD_params, 'cyclic_learning_ex2', True, True)


  0%|          | 0/10 [00:00<?, ?it/s]	 Epoch 0: train_loss = 2.429575599446275, val_loss = 2.4260667079286717,  
 	 train_acc = 0.1015, val_acc = 0.1043
 10%|█         | 1/10 [00:04<00:39,  4.43s/it]	 Epoch 1: train_loss = 2.1722377530693757, val_loss = 2.1860343347670312,  
 	 train_acc = 0.2141, val_acc = 0.2123
 20%|██        | 2/10 [00:09<00:37,  4.71s/it]	 Epoch 2: train_loss = 2.083522398511192, val_loss = 2.107887407643722,  
 	 train_acc = 0.2543, val_acc = 0.2454
 30%|███       | 3/10 [00:14<00:32,  4.69s/it]	 Epoch 3: train_loss = 2.0224570558929607, val_loss = 2.0539571219846073,  
 	 train_acc = 0.2834, val_acc = 0.267
 40%|████      | 4/10 [00:18<00:27,  4.61s/it]	 Epoch 4: train_loss = 1.976261402073833, val_loss = 2.0151068133150765,  
 	 train_acc = 0.3032, val_acc = 0.2864
 50%|█████     | 5/10 [00:23<00:22,  4.52s/it]	 Epoch 5: train_loss = 1.939885992472477, val_loss = 1.985120109740847,  
 	 train_acc = 0.3197, val_acc = 0.2982
 60%|██████    | 6/10 [00:27<00:17,  

In [ ]:
# Load all data 

In [12]:
importlib.reload(m)
X_train_whole, y_train_whole, Y_train_whole = u.load_data('/data_batch_1', clipping=True)
for i in range(2,6):
    X, y, Y = u.load_data('/data_batch_'+str(i), clipping=True)
    X_train_whole = np.concatenate((X, X_train_whole), axis=1)
    y_train_whole = np.concatenate((y, y_train_whole))
    Y_train_whole = np.concatenate((Y, Y_train_whole), axis=1)

X_val_small, y_val_small, Y_val_small = X_train_whole[:,-1000:], y_train_whole[-1000:], Y_train_whole[:,-1000:]
X_train_whole, y_train_whole, Y_train_whole = X_train_whole[:,:-1000], y_train_whole[:-1000], Y_train_whole[:,:-1000]

## normalize with mean and std of train set 
mean = np.mean(X_train_whole, axis=1)
std = np.std(X_train_whole, axis=1)

X_train_whole -= np.outer(mean, np.ones(X_train_whole.shape[1]))
X_train_whole /= np.outer(std, np.ones(X_train_whole.shape[1]))

X_val_small -= np.outer(mean, np.ones(X_val_small.shape[1]))
X_val_small /= np.outer(std, np.ones(X_val_small.shape[1]))

In [14]:
data = {'X_train':X_train_whole, 'Y_train':Y_train_whole, 'y_train':y_train_whole,'X_val':X_val_small, 'Y_val':Y_val_small, 'y_val':y_val_small}

# Do lambda search

In [16]:
importlib.reload(m)
GD_params = {"epochs":2, "n_batch":100, "eta_min":0.001,'eta_max':1e-1, 'ns':2*np.floor(X_train_whole.shape[1]/100), 'n_cycles':1, 'freq':10}
search = m.LambdaSearch(1e-5, 1e-1,10)
model = search.lambda_search(data=data,GDparams=GD_params)

  0%|          | 0/4 [00:08<?, ?it/s]


KeyboardInterrupt: 

In [ ]:
    def plot_history(self,hist,metric):

        if metric == 'loss':
            plt.plot(hist['train_loss'])
            plt.plot(hist['val_loss'])
            plt.xlabel('epoch')
            plt.ylabel('loss')
            plt.legend(['train_loss', 'val_loss'])
        elif metric == 'acc':
            plt.plot(hist['train_acc'])
            plt.plot(hist['val_acc'])
            plt.xlabel('epoch')
            plt.ylabel('accuracy')
            plt.legend(['train_acc', 'val_acc'])
        plt.show()